In [35]:
import yfinance as yf
from sklearn import preprocessing

In [85]:
ticker = "BARC.L"

data1 = yf.download(ticker , start="2015-01-01", end="2025-01-01", auto_adjust=False)

data1.to_csv("barclays_data.csv")

[*********************100%***********************]  1 of 1 completed


In [12]:
ticker = "SHEL.L"

data2 = yf.download(ticker , start="2015-01-01", end="2025-01-01", auto_adjust=False)

data2.to_csv("shell_data.csv")

[*********************100%***********************]  1 of 1 completed


In [13]:
ticker = "HLN.L"

data3 = yf.download(ticker , start="2015-01-01", end="2025-01-01", auto_adjust=False)

data3.to_csv("haleon_data.csv")

[*********************100%***********************]  1 of 1 completed


In [14]:
ticker = "RMV.L"

data4 = yf.download(ticker , start="2015-01-01", end="2025-01-01", auto_adjust=False)

data4.to_csv("rightmove_data.csv")

[*********************100%***********************]  1 of 1 completed


In [15]:
ticker = "EXPN.L"

data5 = yf.download(ticker , start="2015-01-01", end="2025-01-01", auto_adjust=False)

data5.to_csv("experian_data.csv")

[*********************100%***********************]  1 of 1 completed


In [48]:
data1.shape

(2526, 6)

In [49]:
data1.isnull().sum()

Price      Ticker
Adj Close  BARC.L    0
Close      BARC.L    0
High       BARC.L    0
Low        BARC.L    0
Open       BARC.L    0
Volume     BARC.L    0
dtype: int64

In [86]:
data1

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,BARC.L,BARC.L,BARC.L,BARC.L,BARC.L,BARC.L
Date,,,,,,
2015-01-02,242.415970,243.149994,245.600006,241.550003,242.199997,20219711
2015-01-05,233.991486,234.699997,243.149994,234.199997,242.300003,39050852
2015-01-06,229.654602,230.350006,235.850006,230.250000,234.050003,37573308
2015-01-07,229.903870,230.600006,233.460007,230.149994,232.100006,37219841
2015-01-08,236.334412,237.050003,238.350006,231.550003,234.300003,41082776
...,...,...,...,...,...,...
2024-12-23,261.531921,261.600006,262.049988,259.299988,261.200012,13605305
2024-12-24,263.481415,263.549988,264.181000,262.273010,262.950012,8298189


In [87]:
data1.columns = data1.columns.droplevel(1)

In [88]:
data1 = data1.reset_index()

In [89]:
data1

Price,Date,Adj Close,Close,High,Low,Open,Volume
0,2015-01-02,242.415970,243.149994,245.600006,241.550003,242.199997,20219711
1,2015-01-05,233.991486,234.699997,243.149994,234.199997,242.300003,39050852
2,2015-01-06,229.654602,230.350006,235.850006,230.250000,234.050003,37573308
3,2015-01-07,229.903870,230.600006,233.460007,230.149994,232.100006,37219841
4,2015-01-08,236.334412,237.050003,238.350006,231.550003,234.300003,41082776
...,...,...,...,...,...,...,...
2521,2024-12-23,261.531921,261.600006,262.049988,259.299988,261.200012,13605305
2522,2024-12-24,263.481415,263.549988,264.181000,262.273010,262.950012,8298189
2523,2024-12-27,264.831085,264.899994,265.899994,263.540009,264.350006,15970484
2524,2024-12-30,264.681122,264.750000,265.750000,261.399994,262.600006,14724719


In [90]:
data1.drop(["Close", "High", "Low", "Open", "Volume"], axis = 1, inplace = True)

In [91]:
data1

Price,Date,Adj Close
0,2015-01-02,242.415970
1,2015-01-05,233.991486
2,2015-01-06,229.654602
3,2015-01-07,229.903870
4,2015-01-08,236.334412
...,...,...
2521,2024-12-23,261.531921
2522,2024-12-24,263.481415
2523,2024-12-27,264.831085
2524,2024-12-30,264.681122


In [92]:
data1.describe

<bound method NDFrame.describe of Price       Date   Adj Close
0     2015-01-02  242.415970
1     2015-01-05  233.991486
2     2015-01-06  229.654602
3     2015-01-07  229.903870
4     2015-01-08  236.334412
...          ...         ...
2521  2024-12-23  261.531921
2522  2024-12-24  263.481415
2523  2024-12-27  264.831085
2524  2024-12-30  264.681122
2525  2024-12-31  268.080231

[2526 rows x 2 columns]>

In [93]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2526 entries, 0 to 2525
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       2526 non-null   datetime64[ns]
 1   Adj Close  2526 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 39.6 KB


In [34]:
train_size = int(len(data1) * 0.65)
test_size = len(data1) - train_size
train_data, test_data = data1[0:train_size,:], data1[train_size:len(data1),:1]

InvalidIndexError: (slice(0, 1641, None), slice(None, None, None))

In [36]:
scaler = preprocessing.RobustScaler()
scaled_data = scaler.fit_transform(data1)